# Pipeline to get out a csv I can read into R

In [63]:
import pandas as pd
import re
from icecream import ic

In [64]:
#df = pd.read_csv("retweets_GROUP1.csv").drop("retweets", axis = 1)[0:2000]

In [65]:
df = pd.read_csv("TM_output_GROUP1.csv")#[0:2000]

In [66]:
#df["date"] = pd.to_datetime(df["created_at"], utc=True).dt.strftime('%Y-%m-%d')
dict_counts = df['text'].value_counts().to_dict()
counts = pd.DataFrame.from_dict(dict_counts, orient= "index").reset_index().rename(columns={"index": "text",0:"tweet_freq"})

In [67]:
df = pd.merge(df, counts)

In [68]:
# Choose the tweets that get retweeted more than 3 times :)
df = df[df["tweet_freq"] >= 5].reset_index(drop=True)
#df = df[df["tweet_freq"] != 8].reset_index(drop=True)

In [69]:
ic(len(df))
df["topic_nr"] = df["topic_nr"] +1
df.head()

ic| len(df): 183


,created_at,id,text,from_user_id,topics_str,topic_nr,tweet_freq
0,2020-09-04 14:39:33,1.301893e+18,"RT @BroennumKom: Kan I huske, da landsholdet k...",2.219003e+08,"['barn', 'unge', 'mor', 'liv', 'skole', 'menne...",91,5
1,2020-09-04 12:11:12,1.301855e+18,"RT @BroennumKom: Kan I huske, da landsholdet k...",1.208731e+18,"['barn', 'unge', 'mor', 'liv', 'skole', 'menne...",91,5
2,2020-09-04 07:44:20,1.301788e+18,"RT @BroennumKom: Kan I huske, da landsholdet k...",1.212436e+18,"['barn', 'unge', 'mor', 'liv', 'skole', 'menne...",91,5
3,2020-09-04 10:17:41,1.301827e+18,"RT @BroennumKom: Kan I huske, da landsholdet k...",3.641712e+08,"['barn', 'unge', 'mor', 'liv', 'skole', 'menne...",91,5
4,2020-09-04 12:04:38,1.301854e+18,"RT @BroennumKom: Kan I huske, da landsholdet k...",1.183859e+18,"['barn', 'unge', 'mor', 'liv', 'skole', 'menne...",91,5


In [70]:
df.topic_nr.unique() # I need topic numbers to be from 1 to ntopics - not this mess

array([ 91, 105,   4,  19, 164, 133,  92,  65,   3,  46, 158,  43,  77,
        69,   6,  20,  30,  87,  11,   9, 374,   7,  12,  27, 195,  82,
        55,  78,  59,  29,  67])

In [71]:
len(df.topic_nr.unique())

31

In [72]:
# Get the original person who tweeted as "originator"
def get_originator(row):
    return re.findall(r'RT @(.*?)\:' , row["text"])[0]

df["originator"] = df.apply(lambda row: get_originator(row), axis = 1)

In [73]:
# Workaround to get the ids per texts - "document"
df = df.sort_values("text")
newdf = df.drop_duplicates('text').reset_index(drop=True).reset_index()
newdf['document'] = newdf['index']
df = df.drop("topic_nr", axis=1) # this ensures i just have one topic per tweet
newdf = df.merge(newdf, on = "text", how='outer')
newdf = newdf[["created_at_x", "id_x", "text", "from_user_id_x", "originator_x", "document", "topic_nr"]]

In [74]:
# Workaround to get ids per topic in a sensible way - "topic"
newdf = newdf.sort_values("topic_nr")
newdf2 = newdf.drop_duplicates('topic_nr').reset_index(drop=True).reset_index()
newdf2['topic_id'] = newdf2['index']+1
#newdf2['topic_id'] = "user" + newdf2['user_id'].astype(str)
newdf2 = newdf.merge(newdf2, on="topic_nr", how='outer')
newdf2 = newdf2[["created_at_x_x", "originator_x_x", "document_x", "topic_id", "from_user_id_x_x"]]
newdf2.columns = ["created_at", "originator", "document", "topic", "from_user_id"]

newdf2

,created_at,originator,document,topic,from_user_id
0,2020-07-04 10:31:00+00:00,torben180383,33,1,1.208731e+18
1,2020-07-04 10:30:19+00:00,torben180383,33,1,3.641712e+08
2,2020-07-05 07:18:43+00:00,torben180383,33,1,1.060295e+18
3,2020-07-06 12:03:18+00:00,torben180383,33,1,9.595041e+07
4,2020-07-15 21:42:37+00:00,EjnerHarRet,8,1,1.225760e+18
...,...,...,...,...,...
178,2020-03-07 19:19:27+00:00,M_B_Petersen,14,22,2.492950e+07
179,2020-03-07 15:59:39+00:00,M_B_Petersen,14,22,9.170938e+07
180,2020-03-08 08:15:51+00:00,M_B_Petersen,14,22,1.001835e+09
181,2020-03-07 20:27:32+00:00,M_B_Petersen,14,22,3.867775e+07


In [75]:
# Workaround to get ids per user in a sensible way - "user_id"
newdf = newdf2.sort_values("from_user_id")
newdf2 = newdf.drop_duplicates('from_user_id').reset_index(drop=True).reset_index()
newdf2['user_id'] = newdf2['index']+1
newdf2['user_id'] = "user" + newdf2['user_id'].astype(str)
newdf2 = newdf.merge(newdf2, on="from_user_id", how='outer')
newdf2 = newdf2[["created_at_x", "originator_x", "document_x", "user_id", "topic_x"]]
newdf2.columns = ["created_at", "originator", "document", "user_id", "topic"]

In [76]:
newdf2

,created_at,originator,document,user_id,topic
0,2020-02-27 13:11:20+00:00,Skovbruger,18,user1,20
1,2020-03-07 19:19:27+00:00,M_B_Petersen,14,user1,22
2,2020-10-12 13:33:29,Uffe_Tang,23,user1,8
3,2020-08-31 11:28:57,dr2deadline,27,user2,5
4,2020-11-26 15:35:47,SteffenFrolund,19,user2,21
...,...,...,...,...,...
178,2020-07-15 21:42:37+00:00,EjnerHarRet,8,user60,1
179,2020-07-16 13:56:34+00:00,EjnerHarRet,8,user61,1
180,2020-07-15 14:03:32+00:00,jonasjuhler,28,user62,11
181,2020-11-26 07:09:36,SteffenFrolund,19,user63,21


In [79]:
# Lists of documents and usernames
documents = newdf2.document.drop_duplicates().to_list()
users = newdf2.user_id.drop_duplicates().to_list()
topics = newdf2.topic.drop_duplicates().to_list()

In [80]:
ic(len(users))
ic(len(documents))
ic(len(topics))

ic| len(users): 64
ic| len(documents): 34
ic| len(topics): 22


22

In [81]:
usersdf = newdf2.user_id.drop_duplicates().reset_index()
#usersdf['user_id'] = "user" + usersdf['user_id'].astype(str)

df_T = usersdf.transpose().reset_index()
df_T.columns = df_T.iloc[1]
df_T = df_T.drop([0,1]).reset_index()

# Get documents into a long version, repeat rows 8 times (timesteps per document)
doc_df = newdf2[["document", "topic"]].drop_duplicates().reset_index() # Make the df
doc_df = doc_df.loc[doc_df.index.repeat(8)].reset_index(drop=True)

In [82]:
# My empty dataframe with rows as usernames, and rows are timesteps per document
data = pd.concat([df_T, doc_df]).fillna(0).drop(["user_id"], axis = 1)

In [83]:
data

,index,user1,user2,user3,user4,user5,user6,user7,user8,user9,...,user57,user58,user59,user60,user61,user62,user63,user64,document,topic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.0,20.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.0,20.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.0,20.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.0,20.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,18.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.0,6.0
268,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.0,6.0
269,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.0,6.0
270,99,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.0,6.0


In [84]:
# Generate the timesteps: just repeat 0:7 the times we have documents
timesteps = pd.Series(["0,1,2,3,4,5,6,7"]).repeat(len(documents))
timesteps = pd.DataFrame(timesteps).reset_index()
timesteps.columns = ["index","timestep"]

def convert(string):
    string = string["timestep"]
    li = list(string.split(",")) 
    return li

timesteps["timestep"] = timesteps.apply(lambda row: convert(row), axis = 1)

# Times per row
# convert list of pd.Series then stack it
timesteps = (timesteps
     .set_index(['index'])['timestep']
     .apply(pd.Series)
     .stack()
     .reset_index()
     .drop(['level_1', 'index'], axis=1)
     .rename(columns={0:'timestep'}))

data.insert(loc = 1, column= "timestep", value=timesteps.timestep)#["timesteps"] = timesteps

## Fill the empty dataframe!

In [85]:
# Make a results dataframe which has the same columns of data but is empty - will append to it later
results = data[0:0]
results

,index,timestep,user1,user2,user3,user4,user5,user6,user7,user8,...,user57,user58,user59,user60,user61,user62,user63,user64,document,topic


In [86]:
for document in documents:
    df_sub = newdf2[newdf2["document"] == document]
    data_sub = data[data["document"] == document].reset_index(drop=True)
    # Create local timesteps - double reset index because 1. restarts the indexing, 2. restarts indexing and saves that as timestep so it always starts at 0
    df_sub = df_sub.reset_index(drop=True).reset_index().rename(columns={"index":"timestep"})
    usernames = df_sub.user_id.to_list()
    
    local_timesteps = df_sub.timestep.to_list()
    for timestep in local_timesteps:
        #print("Timestep: ", timestep)
        for index, row in data_sub.iterrows():
            if row["timestep"] == str(timestep):
                #for user in usernames:
                user = usernames[timestep]
                data_sub[user][timestep] = 1
    results = pd.concat([results, data_sub])

/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/commando/maris/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [87]:
#results["topic"] = results["index"]
res = results.sort_values(["document", "timestep"]).drop(["timestep", "index"], axis = 1)

# Rearrange columns so the order is: document, topic, user1, user2, ..., usern
cols = res.columns.tolist()
cols = cols[-2:] + cols[:-2]

res = res[cols]
res["document"] = res["document"] + 1

In [88]:
res[res["document"] == 2]

,document,topic,user1,user2,user3,user4,user5,user6,user7,user8,...,user55,user56,user57,user58,user59,user60,user61,user62,user63,user64
0,2.0,19.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,19.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.0,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.0,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2.0,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2.0,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
# Read this out and try the R pipeline on it
res.to_csv("test_results_RT_overand_5.csv", index=False, header = True)

In [90]:
ic(len(res.document.unique()))
ic(len(res.topic.unique()))

topics = res.topic.reset_index()
topics[topics['index'].isin([0])].to_csv("test_topics_RT_overand_5.csv", index = False, header = True)

ic| len(res.document.unique()): 34
ic| len(res.topic.unique()): 22


In [91]:
topics[topics['index'].isin([0])]

,index,topic
0,0,4.0
8,0,19.0
16,0,11.0
24,0,9.0
32,0,17.0
40,0,6.0
48,0,3.0
56,0,13.0
64,0,1.0
72,0,6.0


In [92]:
res["document"].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34.])

In [93]:
res.sort_values("topic")

,document,topic,user1,user2,user3,user4,user5,user6,user7,user8,...,user55,user56,user57,user58,user59,user60,user61,user62,user63,user64
7,34.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,34.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,9.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,9.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,15.0,22.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15.0,22.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,15.0,22.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,22.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
len(res.document.unique())

34

In [95]:
len(res.topic.unique())

22

In [96]:
res[res["document"] == 22]

,document,topic,user1,user2,user3,user4,user5,user6,user7,user8,...,user55,user56,user57,user58,user59,user60,user61,user62,user63,user64
0,22.0,10.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,22.0,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
